<a href="https://colab.research.google.com/github/Femi-90/Machine-Learning-Zoomcamp/blob/main/03-Classification/homework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression


In [ ]:

"""
import re
import openpyxl

def clean_csv_to_excel(input_file, output_file):
    with open(input_file, 'r') as f:
        content = f.read()
    cleaned_content = re.sub(r'\"', '', content)

    rows = [row.split(';') for row in cleaned_content.splitlines()]

    workbook = openpyxl.Workbook()
    sheet = workbook.active

    for row_idx, row in enumerate(rows, start=1):
        for col_idx, value in enumerate(row, start=1):
            sheet.cell(row=row_idx, column=col_idx, value=value)

    workbook.save(output_file)

input_file = '/content/bank-full.csv'  # Path to your original CSV file
output_file = '/content/output.xlsx'  # Path to save the cleaned Excel file
clean_csv_to_excel(input_file, output_file)
"""

'\nimport re\nimport openpyxl\n\ndef clean_csv_to_excel(input_file, output_file):\n    with open(input_file, \'r\') as f:\n        content = f.read()\n    cleaned_content = re.sub(r\'"\', \'\', content)\n\n    rows = [row.split(\';\') for row in cleaned_content.splitlines()]\n\n    workbook = openpyxl.Workbook()\n    sheet = workbook.active\n\n    for row_idx, row in enumerate(rows, start=1):\n        for col_idx, value in enumerate(row, start=1):\n            sheet.cell(row=row_idx, column=col_idx, value=value)\n\n    workbook.save(output_file)\n\ninput_file = \'/content/bank-full.csv\'  # Path to your original CSV file\noutput_file = \'/content/output.xlsx\'  # Path to save the cleaned Excel file\nclean_csv_to_excel(input_file, output_file)\n'

In [ ]:

excel_file = pd.read_excel('/content/output.xlsx')
data = excel_file.to_csv('/content/output.csv', index=False)
df2 = pd.read_csv('/content/output.csv')

In [ ]:
df2

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [ ]:
# categorical Data from the original dataset
categorical_data = df2.select_dtypes(include=['object', 'category']).columns
categorical_data

Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'poutcome', 'y'],
      dtype='object')

In [ ]:
df2 = df2.drop('job', axis=1)

Q 1

In [ ]:
df2['education'].mode()[0]

'secondary'

(Q 2)
Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

In [ ]:
# Extract numerical features
numerical_features = df2.select_dtypes(include=np.number)
# Compute correlation matrix
correlation_matrix = numerical_features.corr()
# Display the correlation matrix
correlation_matrix

,age,balance,day,duration,campaign,pdays,previous
age,1.000000,0.097783,-0.009120,-0.004648,0.004760,-0.023758,0.001288
balance,0.097783,1.000000,0.004503,0.021560,-0.014578,0.003435,0.016674
day,-0.009120,0.004503,1.000000,-0.030206,0.162490,-0.093044,-0.051710
duration,-0.004648,0.021560,-0.030206,1.000000,-0.084570,-0.001565,0.001203
campaign,0.004760,-0.014578,0.162490,-0.084570,1.000000,-0.088628,-0.032855
pdays,-0.023758,0.003435,-0.093044,-0.001565,-0.088628,1.000000,0.454820
previous,0.001288,0.016674,-0.051710,0.001203,-0.032855,0.454820,1.000000


Now we want to encode the y variable.
Let's replace the values yes/no with 1/0.
Split the data

In [ ]:
df2['y'] = df2['y'].map({'yes': 1, 'no': 0})

In [ ]:
X = df2.drop('y', axis=1)  # Assuming 'y' is your target column
y = df2['y']

X.isna().sum()

,0
age,0
marital,0
education,0
default,0
balance,0
housing,0
loan,0
contact,0
day,0
month,0


Split your data in train/val/test sets with 60%/20%/20% distribution.
Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.

In [ ]:

X_train, X_remaining, y_train, y_remaining = train_test_split(X, y, test_size=0.4, random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_remaining, y_remaining, test_size=0.5, random_state=42)

(Q 3)
Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only.
Round the scores to 2 decimals using round(score, 2).


In [ ]:
categorical_features = X_train.select_dtypes(include=['object', 'category']).columns

mi_scores = {}
for feature in categorical_features:

    if X_train[feature].dtype == 'object':
        X_train[feature] = LabelEncoder().fit_transform(X_train[feature])


    mi_score = mutual_info_classif(X_train[[feature]], y_train, discrete_features=True)[0]
    mi_scores[feature] = round(mi_score, 2)

print(mi_scores)

{'marital': 0.0, 'education': 0.0, 'default': 0.0, 'housing': 0.01, 'loan': 0.0, 'contact': 0.01, 'month': 0.02, 'poutcome': 0.03}


Q4


In [ ]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
categorical_features = X_train.select_dtypes(include=['object', 'category']).columns
encoder.fit(X_train[categorical_features])
encoded_features_train = encoder.transform(X_train[categorical_features])
encoded_features_val = encoder.transform(X_val[categorical_features])
encoded_features_test = encoder.transform(X_test[categorical_features])
encoded_feature_names = encoder.get_feature_names_out(categorical_features)

In [ ]:
encoded_features_train_df = pd.DataFrame(encoded_features_train, columns=encoded_feature_names, index=X_train.index)
encoded_features_val_df = pd.DataFrame(encoded_features_val, columns=encoded_feature_names, index=X_val.index)
encoded_features_test_df = pd.DataFrame(encoded_features_test, columns=encoded_feature_names, index=X_test.index)
X_train_encoded = pd.concat([X_train.drop(categorical_features, axis=1), encoded_features_train_df], axis=1)
X_val_encoded = pd.concat([X_val.drop(categorical_features, axis=1), encoded_features_val_df], axis=1)
X_test_encoded = pd.concat([X_test.drop(categorical_features, axis=1), encoded_features_test_df], axis=1)

In [ ]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_encoded, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [ ]:
y_pred = model.predict(X_val_encoded)
accuracy = accuracy_score(y_val, y_pred)
rounded_accuracy = round(accuracy, 2)
print(f"Accuracy on validation set: {rounded_accuracy}")

ValueError: could not convert string to float: 'married'